# GAIA Dataset Analysis
## Understanding the 165 GAIA validation examples

**Objective:** Analyze GAIA patterns and build RAG vector store  
**Output:** Tool priorities and FAISS index for agent development

---

In [ ]:
# Setup and imports
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, OrderedDict
import random
import re
from pathlib import Path
from datetime import datetime
import os
from typing import Dict, List, Optional, Tuple
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

print("🔍 GAIA Dataset Analysis")
print("=" * 40)
print("Goal: Understand patterns and build vector store")
print("=" * 40)

# Section 1: Load & Explore GAIA Data

In [ ]:
def load_gaia_metadata(file_path='metadata.jsonl'):
    """Load and parse GAIA validation dataset"""
    try:
        json_QA = []
        with open(file_path, 'r') as f:
            for line in f:
                json_data = json.loads(line.strip())
                json_QA.append(json_data)
        
        print(f"✅ Successfully loaded {len(json_QA)} GAIA examples")
        return json_QA
    except FileNotFoundError:
        print("❌ metadata.jsonl not found. Creating sample data for demonstration.")
        return create_sample_gaia_data()
    except Exception as e:
        print(f"❌ Error loading GAIA data: {e}")
        return []

def create_sample_gaia_data():
    """Create sample GAIA data for demonstration purposes"""
    sample_data = [
        {
            'task_id': 'sample_001',
            'Question': 'What is the population of Seattle according to the 2020 census?',
            'Level': 1,
            'Final answer': '737015',
            'file_name': None,
            'Annotator Metadata': {
                'Steps': 'Search for Seattle population 2020 census data',
                'Tools': 'web browser\nsearch engine'
            }
        },
        {
            'task_id': 'sample_002', 
            'Question': 'Calculate the compound interest on $5000 at 3.5% annual rate for 10 years',
            'Level': 2,
            'Final answer': '7052.78',
            'file_name': None,
            'Annotator Metadata': {
                'Steps': 'Use compound interest formula: A = P(1 + r)^t',
                'Tools': 'calculator'
            }
        },
        {
            'task_id': 'sample_003',
            'Question': 'What is the average temperature in the attached Excel file?',
            'Level': 1,
            'Final answer': '23.4',
            'file_name': 'temperature_data.xlsx',
            'Annotator Metadata': {
                'Steps': 'Open Excel file, calculate average of temperature column',
                'Tools': 'excel\ncalculator'
            }
        }
    ]
    print("📝 Using sample GAIA data for demonstration")
    return sample_data

# Load the dataset
json_QA = load_gaia_metadata()

In [ ]:
# Basic dataset statistics
if json_QA:
    print(f"\n📈 Dataset Overview:")
    print(f"  ├── Total Questions: {len(json_QA)}")
    
    # Level distribution
    if 'Level' in json_QA[0]:
        levels = [q.get('Level', 'Unknown') for q in json_QA]
        level_counts = Counter(levels)
        print(f"  ├── Level Distribution:")
        for level in sorted(level_counts.keys()):
            print(f"  │   ├── Level {level}: {level_counts[level]} questions")
    
    # File attachment analysis
    files_present = sum(1 for q in json_QA if q.get('file_name'))
    print(f"  ├── Questions with Files: {files_present}")
    print(f"  └── Questions without Files: {len(json_QA) - files_present}")

In [ ]:
def analyze_gaia_patterns(sample_size=5):
    """Analyze question patterns and structures"""
    if not json_QA:
        return
    
    print(f"\n🔍 Sample Question Analysis (Random {sample_size}):")
    print("=" * 60)
    
    sample_questions = random.sample(json_QA, min(sample_size, len(json_QA)))
    
    for i, sample in enumerate(sample_questions, 1):
        print(f"\n📝 Question {i}:")
        print(f"ID: {sample.get('task_id', 'N/A')}")
        print(f"Level: {sample.get('Level', 'N/A')}")
        print(f"Question: {sample.get('Question', 'N/A')[:100]}...")
        print(f"File: {sample.get('file_name', 'None')}")
        
        if 'Annotator Metadata' in sample:
            metadata = sample['Annotator Metadata']
            print(f"Steps: {metadata.get('Steps', 'N/A')[:80]}...")
            print(f"Tools: {metadata.get('Tools', 'N/A')}")
        
        print(f"Answer: {sample.get('Final answer', 'N/A')}")
        print("-" * 40)

# Run pattern analysis
analyze_gaia_patterns()

# Section 2: Tool Usage Analysis

In [ ]:
def analyze_tool_usage():
    """Analyze tool frequency - CRITICAL FOR IMPLEMENTATION PRIORITY"""
    if not json_QA:
        return {}
    
    tools = []
    tool_details = []
    
    for sample in json_QA:
        if 'Annotator Metadata' in sample and 'Tools' in sample['Annotator Metadata']:
            tools_text = sample['Annotator Metadata']['Tools']
            
            # Parse tools (handle different formats)
            tool_lines = tools_text.split('\n')
            for tool_line in tool_lines:
                tool = tool_line.strip()
                if tool.startswith('- '):
                    tool = tool[2:].strip()
                elif tool.startswith('• '):
                    tool = tool[2:].strip()
                
                # Clean up tool names
                tool = tool.lower().strip()
                if tool.startswith('('):
                    end_paren = tool.find(')')
                    if end_paren != -1:
                        tool = tool[end_paren+1:].strip()
                
                if tool and tool != '':
                    tools.append(tool)
                    tool_details.append({
                        'tool': tool,
                        'question_id': sample.get('task_id'),
                        'level': sample.get('Level'),
                        'has_file': sample.get('file_name') is not None
                    })
    
    # Count tool frequencies
    tools_counter = OrderedDict(Counter(tools).most_common())
    
    print("🎯 Tool Usage Priority Analysis:")
    print(f"Total tool instances: {len(tools)}")
    print(f"Unique tools identified: {len(tools_counter)}")
    print("\n📊 Implementation Priority (by frequency):")
    
    for i, (tool, count) in enumerate(tools_counter.items(), 1):
        priority = "🔴 HIGH" if count >= 10 else "🟡 MEDIUM" if count >= 5 else "🟢 LOW"
        print(f"  {i:2d}. {tool:<25} : {count:3d} occurrences {priority}")
    
    return tools_counter, tool_details

# Run tool analysis
tools_counter, tool_details = analyze_tool_usage()

In [ ]:
# Create visualization of tool usage
if tools_counter:
    plt.figure(figsize=(12, 8))
    
    # Top 15 tools
    top_tools = list(tools_counter.items())[:15]
    tool_names = [item[0] for item in top_tools]
    tool_counts = [item[1] for item in top_tools]
    
    colors = ['#FF6B6B' if count >= 10 else '#4ECDC4' if count >= 5 else '#45B7D1' for count in tool_counts]
    
    bars = plt.barh(range(len(tool_names)), tool_counts, color=colors)
    plt.yticks(range(len(tool_names)), tool_names)
    plt.xlabel('Frequency')
    plt.title('GAIA Tool Usage Analysis\n(Implementation Priority Guide)')
    plt.gca().invert_yaxis()
    
    # Add count labels
    for i, (bar, count) in enumerate(zip(bars, tool_counts)):
        plt.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2, 
                str(count), va='center', fontweight='bold')
    
    # Add legend
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='#FF6B6B', label='HIGH Priority (≥10)'),
        Patch(facecolor='#4ECDC4', label='MEDIUM Priority (5-9)'), 
        Patch(facecolor='#45B7D1', label='LOW Priority (<5)')
    ]
    plt.legend(handles=legend_elements, loc='lower right')
    
    plt.tight_layout()
    plt.show()

In [ ]:
def generate_implementation_recommendations(tools_counter):
    """Generate data-driven tool implementation recommendations"""
    if not tools_counter:
        return
    
    print("\n💡 Implementation Recommendations:")
    print("=" * 45)
    
    # Essential tools (high frequency)
    essential = [(tool, count) for tool, count in tools_counter.items() if count >= 10]
    important = [(tool, count) for tool, count in tools_counter.items() if 5 <= count < 10]
    optional = [(tool, count) for tool, count in tools_counter.items() if count < 5]
    
    print(f"🔴 ESSENTIAL TOOLS (Implement First):")
    for tool, count in essential:
        print(f"  ├── {tool}: {count} occurrences")
    
    print(f"\n🟡 IMPORTANT TOOLS (Implement Second):")
    for tool, count in important:
        print(f"  ├── {tool}: {count} occurrences")
    
    print(f"\n🟢 OPTIONAL TOOLS (If Budget Allows):")
    for tool, count in optional[:5]:  # Show top 5 optional
        print(f"  ├── {tool}: {count} occurrences")
    
    # File type analysis
    print(f"\n📁 File Processing Requirements:")
    file_questions = [q for q in json_QA if q.get('file_name')]
    if file_questions:
        file_extensions = []
        for q in file_questions:
            filename = q.get('file_name', '')
            if '.' in filename:
                ext = Path(filename).suffix.lower()
                file_extensions.append(ext)
        
        ext_counts = Counter(file_extensions)
        for ext, count in ext_counts.most_common():
            print(f"  ├── {ext}: {count} files")

# Generate recommendations
generate_implementation_recommendations(tools_counter)

# Section 3: Build FAISS Vector Store

In [ ]:
def setup_simple_faiss_store():
    """Simple FAISS setup for RAG"""
    
    print("🔧 Setting up FAISS Vector Store:")
    print("  ├── In-memory for speed")
    print("  ├── Disk persistence")
    print("  └── Simple reset = delete files")
    
    return {
        "index_file": "gaia_examples.faiss",
        "metadata_file": "gaia_examples_meta.json",
        "embedding_model": "sentence-transformers/all-mpnet-base-v2"
    }

def reset_faiss_store():
    """Clean slate - delete existing files"""
    files = ["gaia_examples.faiss", "gaia_examples_meta.json"]
    deleted = []
    
    for file_path in files:
        if os.path.exists(file_path):
            os.remove(file_path)
            deleted.append(file_path)
    
    if deleted:
        print(f"🗑️  Deleted: {', '.join(deleted)}")
    else:
        print("✅ No existing files to delete")
    
    return True

# Setup FAISS configuration
config = setup_simple_faiss_store()

In [ ]:
def create_faiss_index():
    """Create FAISS index with GAIA examples"""
    
    if not json_QA:
        print("❌ No GAIA data available")
        return None
        
    print(f"\n📚 Creating FAISS index with {len(json_QA)} examples...")
    
    # Document preparation
    docs = []
    for sample in json_QA:
        content = f"Question: {sample.get('Question', '')}\nAnswer: {sample.get('Final answer', '')}"
        
        # Add steps if available
        if 'Annotator Metadata' in sample and 'Steps' in sample['Annotator Metadata']:
            content += f"\nSteps: {sample['Annotator Metadata']['Steps']}"
            
        docs.append({
            'id': sample.get('task_id'),
            'content': content,
            'level': sample.get('Level'),
            'has_file': sample.get('file_name') is not None
        })
    
    print(f"  ├── Prepared {len(docs)} documents")
    print(f"  ├── Average length: {np.mean([len(d['content']) for d in docs]):.0f} chars")
    
    # For production, uncomment this:
    """
    from sentence_transformers import SentenceTransformer
    import faiss
    
    # Generate embeddings
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    texts = [doc['content'] for doc in docs]
    embeddings = model.encode(texts)
    
    # Create FAISS index
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    faiss.normalize_L2(embeddings)
    index.add(embeddings.astype('float32'))
    
    # Save to disk
    faiss.write_index(index, "gaia_examples.faiss")
    
    # Save metadata
    metadata = [{'id': doc['id'], 'level': doc['level'], 'has_file': doc['has_file']} for doc in docs]
    with open("gaia_examples_meta.json", "w") as f:
        json.dump(metadata, f)
    
    print(f"✅ FAISS index saved with {index.ntotal} vectors")
    """
    
    print(f"  ├── Would generate embeddings (768-dim)")
    print(f"  ├── Would create FAISS IndexFlatIP")
    print(f"  ├── Would save to: gaia_examples.faiss")
    print(f"  └── Would save metadata: gaia_examples_meta.json")
    
    return len(docs)

# Reset and create index
print("1️⃣  Reset existing store:")
reset_success = reset_faiss_store()

print("\n2️⃣  Create FAISS index:")
if reset_success:
    index_size = create_faiss_index()

In [ ]:
def test_search_simulation():
    """Simulate how FAISS search would work"""
    
    test_queries = [
        "What is the population of a city?",
        "Calculate compound interest", 
        "Find information in spreadsheet",
        "Search for scientific information"
    ]
    
    print("\n🔍 Testing Search Simulation:")
    print("=" * 40)
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n{i}. Query: '{query}'")
        
        # Simulate results from similar GAIA examples
        mock_results = []
        for j, sample in enumerate(json_QA[:2]):
            mock_results.append({
                'score': random.uniform(0.7, 0.95),
                'question': sample.get('Question', '')[:50] + '...',
                'answer': sample.get('Final answer', ''),
                'level': sample.get('Level')
            })
        
        print("📊 Similar Examples Found:")
        for j, result in enumerate(mock_results, 1):
            print(f"  {j}. [{result['score']:.3f}] {result['question']}")
            print(f"     Answer: {result['answer']}")
        
        # Show agent routing suggestion
        if "calculate" in query.lower():
            agent = "data_analyst"
        elif "search" in query.lower() or "population" in query.lower():
            agent = "web_researcher"
        elif "spreadsheet" in query.lower():
            agent = "data_analyst"
        else:
            agent = "general_assistant"
            
        print(f"   🤖 Suggested agent: {agent}")

# Test search simulation
test_search_simulation()

print("\n" + "="*50)
print("🎉 DATASET ANALYSIS COMPLETE!")
print("="*50)
print("✅ Tool priorities identified")
print("✅ File type analysis done")
print("✅ FAISS vector store designed")
print("✅ Ready for agent development")
print("="*50)